https://robotics.stackexchange.com/questions/18129/inverse-kinematics-problem-of-articulated-manipulators-arm

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random
import tensorflow as tf
# tf.test.gpu_device_name()

2022-11-15 15:59:23.390879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 15:59:23.481201: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-15 15:59:23.499209: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 15:59:23.795355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [3]:
X_train = np.genfromtxt('X_train.csv',delimiter=',')
Y_train = np.genfromtxt('Y_train.csv',delimiter=',')

In [4]:
print(X_train.shape,Y_train.shape)

(3, 5000) (3, 5000)


In [5]:
X_train = X_train.T
Y_train = Y_train.T
print(X_train.shape,Y_train.shape)

(5000, 3) (5000, 3)


In [19]:
print(X_train[1],Y_train[1])

[ 0.740985   -0.57896637 -0.03206151] [1.6140994  1.95241047 1.12820251]


## Create Model

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import numpy
from keras import backend as K
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

from IPython.display import clear_output

In [7]:
def base_model():
     model = Sequential()
     model.add(Dense(32, input_dim=3, activation='relu'))
     model.add(Dense(64, activation='relu'))
     model.add(Dense(128, activation='relu'))
     model.add(Dense(256, activation='relu'))
     model.add(Dense(128, activation='relu'))
     model.add(Dense(32, activation='relu'))
     model.add(Dense(3, activation = 'linear'))
     model.compile(loss='mean_absolute_error', optimizer = 'adam')
     return model
clf = KerasRegressor(build_fn=base_model, epochs=50000, batch_size=5000,verbose=0)

/tmp/ipykernel_14210/960966372.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasRegressor(build_fn=base_model, epochs=50000, batch_size=5000,verbose=0)


In [8]:
print(base_model().summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                128       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 256)               33024     
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 32)                4128      
                                                                 
 dense_6 (Dense)             (None, 3)                 9

2022-11-15 15:59:33.795925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 15:59:33.816190: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 15:59:33.816299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 15:59:33.816687: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [9]:
print("STARTING TO TRAIN")
clf.fit(X_train,Y_train)
print("FINISHED  TRAIN")

res = clf.predict(X_train)
print(res)

STARTING TO TRAIN


2022-11-15 15:59:49.390618: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


FINISHED  TRAIN
[[-2.0593321   0.11269358  0.48097473]
 [ 1.754333    1.9818568   0.8447996 ]
 [ 0.73534393 -0.5680857   1.1007854 ]
 ...
 [ 0.46983743 -2.7318013   2.5860078 ]
 [ 0.74842936 -0.2829492   0.14002109]
 [-1.647037   -2.6727786  -2.6739786 ]]


In [10]:
Loss = mean_absolute_error(Y_train, res)
print("Loss {}".format(Loss))

Loss 0.5800576917162981


In [11]:
temp = np.abs(np.abs(res) - np.abs(Y_train))
temp = (temp < 0.2)

In [12]:
temp = temp.astype(int)
print(np.sum(temp)/15000)

0.4514666666666667


## Save Model

In [14]:
clf.save('./saveModel.h5')
clf = keras.models.load_model('./saveModel.h5')

AttributeError: 'KerasRegressor' object has no attribute 'save'

## Forward Kinematic

<img src="./DH3DOF.png" alt="Drawing" style="width: 500px;"/><img src="./kine.png" alt="Drawing" style="width: 700px;"/>

In [29]:
#X=[ 0.740985   -0.57896637 -0.03206151] ,Y=[1.6140994  1.95241047 1.12820251]
result = clf.predict(np.array([[ 0.740985],[-0.57896637],[-0.03206151]]).T)
print(result)

[1.7590027  1.9838214  0.85015035]
